In [40]:
import pandas as pd

demog_df = pd.read_csv(r'C:\Users\MichalinaJanik\PycharmProjects\coding-fairness\Demographic_Features\data\all_features_plus_neighbours.csv', )

topo_df = pd.read_csv(r'C:\Users\MichalinaJanik\PycharmProjects\coding-fairness\topology_based_approach\data\topology_based_features.csv')

pure_demog = pd.read_csv(r'C:\Users\MichalinaJanik\PycharmProjects\coding-fairness\Demographic_Features\data\pure_demographical_data.csv')

topo_cols_to_del = ['Column1','Question', 'coding_success']
topo_identifiers = ['egoid','SurveyNr']


topo_df.drop(columns = topo_cols_to_del, inplace = True)
topo_df.rename(columns = {'EgoID': 'egoid'}, inplace = True)

demog_and_pure_demog =  demog_df.merge(pure_demog, how = 'left', on = topo_identifiers)
demog_and_topo_df = demog_and_pure_demog.merge(topo_df, how = 'left', on = topo_identifiers)


demog_df_copu = demog_df.copy()
demog_df = demog_and_topo_df


to_delete_opinion_columns = ['abortion', 'premaritalsex', 'euthanasia',
       'homosexual', 'deathpen', 'marijuana', 'eqchances', 'fssocsec','toomucheqrights', 'Unnamed: 0']

demog_df.drop(columns=to_delete_opinion_columns, inplace=True)



In [41]:
print(demog_df.columns)

Index(['egoid', 'SurveyNr', 'Discussed_Politics', 'Discussed_Religion',
       'Performed_Volunteer_Work', 'Felt_Depressed', 'timeperweekactivea',
       'Drank_Beer', 'timeperweekactiveb', 'Exercised', 'Drank_Wine_or_Liquor',
       'Felt_Overwhelmed', 'Political_Campaign_Work', 'Public_Communication',
       'Religious_Attendance', 'abstinent', 'Potential_College_Transfer_Score',
       'Likelihood_Student_Gov_Participation_Score',
       'Experiences_of_Depression_Score', 'Future_Academic_Changes_Score',
       'Likelihood_Sports_Participation_Score',
       'Active_Community_Volunteerism_Score', 'Artistic Achievement_Score',
       'Professional Development_Score', 'Family and Personal Life_Score',
       'Classical_Traditional_Score', 'Modern_Popular_Score',
       'Folk_Cultural_Score', 'Nostalgic_Easy_Score', 'Musical_Theater_Score',
       'Music_Diversity_Score', 'Classical_Traditional_Preferred',
       'Modern_Popular_Preferred', 'Nostalgic_Easy_Preferred', 'Fiction_Score',


In [42]:
import pandas as pd
import glob

# Define the path pattern
path_pattern = r"C:\Users\MichalinaJanik\PycharmProjects\coding-fairness\dataset\simulationResults\best_sim_result_2\*.csv"

questions_opinions = ['euthanasia', 'fssocsec', 'fswelfare', 'jobguar', 'marijuana', 'toomucheqrights']

# Initialize a dictionary to store file paths by topic
files_by_topic = {topic: [] for topic in questions_opinions}


import os
for file_path in glob.glob(path_pattern):
    for topic in questions_opinions:
        if topic in os.path.basename(file_path):
            files_by_topic[topic].append(file_path)


data_frames_by_topic = {}
# Specify the columns to extract
columns = ['StudentID', 'SurveyNr', 'OpinionSim', 'OpinionSurvey']


for topic, files in files_by_topic.items():
    topic_data_frames = []
    for f in files:
        try:
            df = pd.read_csv(f, delimiter=';', usecols=lambda column: column in columns)
            if not df.empty:
                topic_data_frames.append(df)
        except ValueError as e:
            print(f"Error reading {f}: {e}")
        except Exception as e:
            print(f"Unexpected error with file {f}: {e}")

    # Concatenate all data frames for the current topic
    if topic_data_frames:
        concatenated_df = pd.concat(topic_data_frames, ignore_index=True)
        concatenated_df.rename(columns={'StudentID': 'egoid'}, inplace=True)
        data_frames_by_topic[topic] = concatenated_df
        # Optionally, save each DataFrame to a CSV file
        concatenated_df.to_csv(f"{topic}_data.csv", index=False)


demog_df = demog_df.reset_index(drop=True)

demog_df['egoid'] = demog_df['egoid'].astype('int64')
demog_df['SurveyNr'] = demog_df['SurveyNr'].astype('int64')


from sklearn.model_selection import train_test_split
import pandas as pd

def generate_datasets_per_topic(topic, demog, minorities, test_size=0.2):
    columns_to_del = ['OpinionSim', 'OpinionSurvey', 'egoid', 'SurveyNr']
    topic_data = data_frames_by_topic.get(topic)
    if topic_data is None:
        return None  # If no data is found for the topic

    topic_data.drop_duplicates(inplace=True)
    demog.drop_duplicates(inplace=True)

    merged = topic_data.merge(demog, on=['egoid', 'SurveyNr'], how='inner')
    merged['Y'] = (merged['OpinionSim'] != merged['OpinionSurvey']).astype(int)

    # Extract all unique minority EgoIDs
    minority_ids = pd.concat(
        [
            (df if isinstance(df, pd.DataFrame) else df.to_frame('egoid')).rename(columns={'EgoID': 'egoid'})
            for df in minorities.values()
        ]
    )['egoid'].unique()

    # Identify minority and non-minority data
    minority_data = merged[merged['egoid'].isin(minority_ids)]
    non_minority_data = merged[~merged['egoid'].isin(minority_ids)]

    # Split minority and non-minority data separately to ensure representation
    minority_train, minority_test = train_test_split(
        minority_data,
        test_size=test_size,
        random_state=42
    )
    non_minority_train, non_minority_test = train_test_split(
        non_minority_data,
        test_size=test_size,
        random_state=42
    )

    # Concatenate the splits to form the full training and testing sets
    X_train = pd.concat([minority_train.drop(columns=['Y']), non_minority_train.drop(columns=['Y'])])
    X_test = pd.concat([minority_test.drop(columns=['Y']), non_minority_test.drop(columns=['Y'])])
    y_train = pd.concat([minority_train['Y'], non_minority_train['Y']])
    y_test = pd.concat([minority_test['Y'], non_minority_test['Y']])

    X_train = X_train.drop(columns=columns_to_del)
    X_test = X_test.drop(columns=columns_to_del)
    # Extracting minority data for evaluation with 'egoid'
    X_minorities_test = minority_test.drop(columns=['Y', 'OpinionSim', 'OpinionSurvey'])
    y_minorities_test = minority_test[['Y', 'egoid']]  # Keep 'egoid' with 'Y'
    X_minorities_test = pd.DataFrame(X_minorities_test)
    X_minorities_test.drop(columns=['SurveyNr'], inplace=True)
    return X_train, X_test, y_train, y_test, X_minorities_test, y_minorities_test






Here is the part where we start to train on prepared and loaded from file data

In [43]:
import pandas as pd
from sklearn.metrics import f1_score

def evaluate_on_minorities_X_y(model, features, X, y, minorities):
    results = []  # Initialize results list here to store the results for each category

    for category, series in minorities.items():
        # Convert series to DataFrame if not already and ensure column names are correct
        df = series if isinstance(series, pd.DataFrame) else pd.DataFrame(series)
        df.rename(columns={'EgoID': 'egoid'}, inplace=True)

        # Get unique identifiers for the minority group
        minority_ids = df['egoid'].unique()

        # Filter X and y for the current minority group based on 'egoid'
        filtered_X = X[X['egoid'].isin(minority_ids)]
        filtered_y = y[y['egoid'].isin(minority_ids)]['Y']  # Assumes 'Y' is the target column in 'y'

        # Ensure only the relevant features are used for prediction
        filtered_X = filtered_X[features]

        if not filtered_X.empty:
            # Make predictions and calculate F1 score
            predictions = model.predict(filtered_X)
            f1 = f1_score(filtered_y, predictions, average='macro')
            results.append({'Category': category, 'F1 Score': f1})
        else:
            results.append({'Category': category, 'F1 Score': 'No Data'})

    return pd.DataFrame(results)


In [44]:
# Import necessary libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from skopt import BayesSearchCV
from sklearn.metrics import make_scorer, f1_score
from joblib import dump, load
import pandas as pd
from skopt.space import Integer

minorities_path = r'C:\Users\MichalinaJanik\PycharmProjects\coding-fairness\dataset\dictionary_of_dfs_with_minorities.pkl'
with open(minorities_path, 'rb') as file:
    minorities = load(file)

# Define parameter grid
param_grid = {
    'n_estimators': Integer(50,400),
    'max_features': ['sqrt', 'log2'],
    'max_depth': (2, 8),
    'min_samples_split': (4, 8),
    'min_samples_leaf': (2, 4),
    "bootstrap": [True, False],
    "criterion": ["gini", "entropy"]
}

select_top = [15, 20, 30, 40, 50, 60, 70, 85]
results_per_topic = {}
best_for_topic = {}

for topic in questions_opinions:
    print(f"Processing topic: {topic}")
    X_train, X_test, y_train, y_test, X_minor, y_minor= generate_datasets_per_topic(topic, demog_df, minorities)
    clf = RandomForestClassifier(class_weight='balanced', random_state=42)
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    scoring_metric = make_scorer(f1_score)
    grid_search = BayesSearchCV(estimator=clf, search_spaces=param_grid, cv=cv, scoring=scoring_metric, n_jobs=-1, verbose=4, n_iter=30)
    grid_search.fit(X_train, y_train)
    print(f"Bayes search completed for topic: {topic}")

    # Extract the best model and its feature importances
    best_model = grid_search.best_estimator_
    feature_importances = best_model.feature_importances_
    sorted_features = [x for _, x in sorted(zip(feature_importances, X_train.columns), reverse=True)]
    best_params = grid_search.best_params_

    # Store results for each subset of features
    best_f1 = 0  # Initialize the best F1 score for comparison
    best_features_subset = None
    results = []
    best_model = None
    for no_features in select_top:
        print(f"Evaluating top {no_features} features for topic: {topic}")
        selected_features = sorted_features[:no_features]
        X_train_sub = X_train[selected_features]
        X_test_sub = X_test[selected_features]


        model = RandomForestClassifier(**best_params, random_state=42, class_weight='balanced')
        model.fit(X_train_sub, y_train)

        # Evaluate and record results
        y_test_pred = model.predict(X_test_sub)
        f1 = f1_score(y_test, y_test_pred)
        results.append({'Top Features': no_features, 'F1 Score': f1})

        # Update if the current model is the best
        if f1 > best_f1:
            print(f"New best model with F1 {f1} using top {no_features} features for topic: {topic}")
            best_f1 = f1
            best_features_subset = selected_features
            best_model = model
            best_model_path = f'C:/Users/MichalinaJanik/PycharmProjects/coding-fairness/Demographic_Features/models/{topic}_best_model_rf_for_selected_features.joblib'
            dump(model, best_model_path)



    results_df = evaluate_on_minorities_X_y(best_model, best_features_subset, X_minor, y_minor, minorities)
    results_per_topic[topic] = results_df

    print(f"Completed processing for topic: {topic}. Best F1: {best_f1}")
    # Store the best results for the topic
    best_for_topic[topic] = {
        'Results DataFrame': pd.DataFrame(results),
        'Best F1 Score': best_f1,
        'Best Features Subset': best_features_subset,
        'Best Model Path': best_model_path,
        'Best params': best_params,
        'minorities results:': results_df
    }

    # Optionally, print or save the evaluation results
    print(f"Results for {topic}:\n{results_df}\n")
    results_df.to_csv(f'C:/Users/MichalinaJanik/PycharmProjects/coding-fairness/Demographic_Features/evaluation_results_{topic}.csv', index=False)

    dump(best_for_topic, r'C:/Users/MichalinaJanik/PycharmProjects/coding-fairness/Demographic_Features/best_for_topic.joblib')
    print(best_for_topic)


Processing topic: euthanasia
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits


C:\Users\MichalinaJanik\AppData\Roaming\Python\Python312\site-packages\numpy\ma\core.py:2892: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits


C:\Users\MichalinaJanik\AppData\Roaming\Python\Python312\site-packages\numpy\ma\core.py:2892: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Bayes search completed for topic: euthanasia
Evaluating top 15 features for topic: euthanasia
New best model with F1 0.7453416149068323 using top 

...

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Bayes search completed for topic: marijuana
Evaluating top 15 features for topic: marijuana
New best model with F1 0.6258503401360545 using top 15 features for topic: marijuana
Evaluating top 20 features for topic: marijuana
Evaluating top 30 features for topic: marijuana
New best model with F1 0.6530612244897959 using top 30 features for topic: marijuana
Evaluating top 40 features for topic: marijuana
New best model with F1 0.6575342465753424 using top 40 features for topic: marijuana
Evaluating top 50 features for topic: marijuana
Evaluating top 60 features for topic: marijuana
New best model with F1 0.6666666666666666 using top 60 features for topic: marijuana
Evaluating top 70 features for topic: marijuana
New best model with F1 0.684931506849315 using top 70 features for topic: marijuana
Evaluating top 85 features for topic: marijuana
New best model with F1 0.6

C:\Users\MichalinaJanik\AppData\Roaming\Python\Python312\site-packages\numpy\ma\core.py:2892: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Fitting 10 folds for each of 1 candidates, totalling 10 fits


C:\Users\MichalinaJanik\AppData\Roaming\Python\Python312\site-packages\numpy\ma\core.py:2892: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Bayes search completed for topic: toomucheqrights
Evaluating top 15 features for topic: toomucheqrights
New best model with F1 0.6792452830188679 using top 15 features for topic: toomucheqrights
Evaluating top 20 features for topic: toomucheqrights
New best model with F1 0.7044025157232704 using top 20 features for topic: toomucheqrights
Evaluating top 30 features for topic: toomucheqrights
New best model with F1 0.7195121951219512 using top 30 features for topic: toomucheqrights
Evaluating top 40 features for topic: toomucheqrights
Evaluating top 50 features for topic: toomucheqrights
New best model with F1 0.7272727272727273 using top 50 features for topic: toomucheqrights
Evaluating top 60 features for topic: toomucheqrights
Evaluating top 70 features for topic: toomucheqrights
New best model with F1 0.7305389221556886 using top 70 features for topic: toomucheqrights
Evaluating top 85 features for topic: toomucheqrights
Co

Evaluating f1 score for minorities for each topic

In [45]:
from joblib import load
import pandas as pd

# Load the data
best_for_topic = load(r'C:/Users/MichalinaJanik/PycharmProjects/coding-fairness/Demographic_Features/best_for_topic.joblib')

for topic in questions_opinions:
    print(f"Results for {topic}: ")
    print(best_for_topic[topic])




Results for euthanasia: 
{'Results DataFrame':    Top Features  F1 Score
0            15  0.745342
1            20  0.756410
2            30  0.766234
3            40  0.751592
4            50  0.767296
5            60  0.784810
6            70  0.787097
7            85  0.795031, 'Best F1 Score': np.float64(0.7950310559006211), 'Best Features Subset': ['Pagerank', 'Religious_Attendance', 'extraversion', 'major', 'Sum of CogsNet', 'Betweenness-centrality', 'Subgraph centrality exp', 'Betweenness Centrality_Survey', 'Load', 'agreeableness', 'gender', 'openness', 'Eigenvector centrality', "Avg Neighbour's CogsNetSum", 'Avg Neighbour Degree', 'Laplacian', 'neuroticism', 'Current flow closeness centrality', 'Degree-centrality', 'Closeness centrality', 'Closeness Centrality_Survey', 'conscientiousness', 'Degree Centrality_Survey', 'Community_Survey', 'heighttotal', 'Family and Personal Life_Score', 'Nostalgic_Easy_Score', 'Likelihood_Student_Gov_Participation_Score', 'Felt_Depressed', 'Disc

In [99]:

import pandas as pd
from joblib import load
from sklearn.metrics import f1_score

def load_model_features(topic):
    # Load the best model for the topic
    model_path = f'C:/Users/MichalinaJanik/PycharmProjects/coding-fairness/Demographic_Features/models/models_per_topic/{topic}_best_model_rf_for_selected_features.joblib'
    model = load(model_path)

    # Load the best features subset, assuming they were saved to CSV
    best_features = best_for_topic[topic]
    best_features_list = best_features['Best Features Subset']

    return model, best_features_list




def evaluate_on_minorities(model, features, dt, minorities):
    results = []
    for category, series in minorities.items():
        df = pd.DataFrame(series)

        print("Available columns in all_data:", dt.columns)
        print("Available columns in category DataFrame:", df.columns)

        # Ensure 'EgoID' is present in both DataFrames
        assert 'EgoID' in dt.columns, "EgoID is missing from all_data"
        assert 'EgoID' in df.columns, "EgoID is missing from the category DataFrame"
        # Filter data for the current category using egoid
        data = dt[dt['EgoID'].isin(df['EgoID'])]
        print('HERE WE GOT ERROR')
        print("Features:")
        elements_to_remove= ['EgoID', 'Y']
        filtered_features = [feature for feature in features if feature not in elements_to_remove]
        print(features)
        print(data.columns)
        X = data[filtered_features]
        y = data['Y'].astype(int)

        # Predict and calculate F1-score
        predictions = model.predict(X)
        f1 = f1_score(y, predictions, average='macro')  # Adjust average parameter as needed

        # Store results
        results.append({'Category': category, 'F1 Score': f1})

    return pd.DataFrame(results)

In [100]:
results_per_topic = {}

for topic in questions_opinions:
    model, features = load_model_features(topic)
    if 'EgoID' not in features:
        features = features + ['EgoID', 'Y']
    topic_data = data_frames_by_topic.get(topic)
    topic_data.rename(columns={'egoid': 'EgoID'}, inplace=True)
    print(topic_data.columns)
    topic_data.drop_duplicates(inplace=True)
    all_data.drop_duplicates(inplace=True)

    merged = topic_data.merge(all_data, on=['EgoID', 'SurveyNr'], how='inner')
    merged['Y'] = (merged['OpinionSim'] != merged['OpinionSurvey']).astype(int)

    # Drop columns without using inplace=True and correctly assign the result to X
    datas = merged.drop(columns=['SurveyNr', 'OpinionSim', 'OpinionSurvey'])

    print(datas.columns)
    results_df = evaluate_on_minorities(model, features, datas, minorities)
    results_per_topic[topic] = results_df

    # Optionally, print or save the evaluation results
    print(f"Results for {topic}:\n{results_df}\n")
    results_df.to_csv(f'C:/Users/MichalinaJanik/PycharmProjects/coding-fairness/Demographic_Features/evaluation_results_{topic}.csv', index=False)


Index(['EgoID', 'SurveyNr', 'OpinionSim', 'OpinionSurvey'], dtype='object')
Index(['EgoID', 'Discussed_Politics', 'Discussed_Religion',
       'Performed_Volunteer_Work', 'Felt_Depressed', 'timeperweekactivea',
       'Drank_Beer', 'timeperweekactiveb', 'Exercised', 'Drank_Wine_or_Liquor',
       'Felt_Overwhelmed', 'Political_Campaign_Work', 'Public_Communication',
       'Religious_Attendance', 'abstinent', 'Potential_College_Transfer_Score',
       'Likelihood_Student_Gov_Participation_Score',
       'Experiences_of_Depression_Score', 'Future_Academic_Changes_Score',
       'Likelihood_Sports_Participation_Score',
       'Active_Community_Volunteerism_Score', 'Artistic Achievement_Score',
       'Professional Development_Score', 'Family and Personal Life_Score',
       'Classical_Traditional_Score', 'Modern_Popular_Score',
       'Folk_Cultural_Score', 'Nostalgic_Easy_Score', 'Musical_Theater_Score',
       'Music_Diversity_Score', 'Classical_Traditional_Preferred',
       'Modern_P